## Vectorization

In [2]:
!pip install torchtext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.6 MB/s eta 0:00:0000:0100:01


In [3]:
from torchtext.data.utils import get_tokenizer

sample1 = 'We are learning AI'
sample2 = 'AI is a CS topic'

# Define tokenizer function
tokenizer = get_tokenizer('basic_english')
sample1_tokens = tokenizer(sample1)
sample2_tokens = tokenizer(sample2)

print(sample1_tokens)
print(sample2_tokens)

['we', 'are', 'learning', 'ai']
['ai', 'is', 'a', 'cs', 'topic']


/Users/quangvinh/miniconda3/envs/torchenv/lib/python3.12/site-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [16]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

sample1 = 'We are learning AI'
sample2 = 'AI is a CS topic'
data = [sample1, sample2]

# Create a function to yield list of tokens
tokenizer = get_tokenizer('basic_english')
def yield_tokens(examples):
    for text in examples:
        yield tokenizer(text)

# Create vocabulary
vocab_size = 8
vocab = build_vocab_from_iterator(yield_tokens(data),
                                  max_tokens=vocab_size,
                                  specials=["<unk>", 
                                            "<pad>"])
vocab.set_default_index(vocab["<unk>"])

In [5]:
vocab.get_stoi()

{'learning': 7,
 'is': 6,
 'are': 4,
 'a': 3,
 'cs': 5,
 'ai': 2,
 '<pad>': 1,
 '<unk>': 0}

In [6]:
tokens = tokenizer(sample1)
print(tokens) 

sample1_tokens = [vocab[token] for token in tokens]
print(sample1_tokens)

['we', 'are', 'learning', 'ai']
[0, 4, 7, 2]


In [7]:
tokens = tokenizer(sample2)
print(tokens) 

sample2_tokens = [vocab[token] for token in tokens]
print(sample2_tokens)

['ai', 'is', 'a', 'cs', 'topic']
[2, 6, 3, 5, 0]


In [10]:
import torch

def vectorize(text, vocab, seq_len):
    tokens = tokenizer(text)    
    tokens = [vocab[token] for token in tokens] 
    
    num_pads = sequence_length - len(tokens)
    tokens = tokens[:sequence_length] + [vocab["<pad>"]]*num_pads
    
    return torch.tensor(tokens, dtype=torch.long)

# Vectorize the samples
sequence_length = 5
vectorized_sample1 = vectorize(sample1, vocab, 
                               sequence_length)
vectorized_sample2 = vectorize(sample2, vocab, 
                               sequence_length)

print("Vectorized Sample 1:", vectorized_sample1)
print("Vectorized Sample 2:", vectorized_sample2)

Vectorized Sample 1: tensor([0, 4, 7, 2, 1])
Vectorized Sample 2: tensor([2, 6, 3, 5, 0])


In [11]:
sample3 = 'AI topic in CS is difficult'
vectorized_sample3 = vectorize(sample3, vocab, 
                               sequence_length)
print(vectorized_sample3)

tensor([2, 0, 0, 5, 6])


## Embedding

In [12]:
import torch
import torch.nn as nn

vocab_size = 8
embed_dim = 4
embedding = nn.Embedding(vocab_size, embed_dim)

custom_weights = torch.tensor( [[-0.1882,  0.5530,  1.6267,  0.7013],
                                [ 1.7840, -0.8278, -0.2701,  1.3586],
                                [ 1.0281, -1.9094,  0.3182,  0.4211],
                                [-1.3083, -0.0987,  0.7647, -0.3680],
                                [ 0.2293,  1.3255,  0.1318,  2.0501],
                                [ 0.4058, -0.6624, -0.8745,  0.7203],
                                [ 0.5582,  0.0786, -0.6817,  0.6902],
                                [ 0.4309, -1.3067, -0.8823,  1.5977]]).float()
embedding.weight = nn.Parameter(custom_weights)
print(embedding.weight)

Parameter containing:
tensor([[-0.1882,  0.5530,  1.6267,  0.7013],
        [ 1.7840, -0.8278, -0.2701,  1.3586],
        [ 1.0281, -1.9094,  0.3182,  0.4211],
        [-1.3083, -0.0987,  0.7647, -0.3680],
        [ 0.2293,  1.3255,  0.1318,  2.0501],
        [ 0.4058, -0.6624, -0.8745,  0.7203],
        [ 0.5582,  0.0786, -0.6817,  0.6902],
        [ 0.4309, -1.3067, -0.8823,  1.5977]], requires_grad=True)


## Update

In [13]:
import torch
import torch.nn as nn

vocab_size = 8
embed_dim = 4
embedding = nn.Embedding(vocab_size, 
                         embed_dim)

custom_weights = torch.tensor( [[-0.1882,  0.5530,  1.6267,  0.7013],
                                [ 1.7840, -0.8278, -0.2701,  1.3586],
                                [ 1.0281, -1.9094,  0.3182,  0.4211],
                                [-1.3083, -0.0987,  0.7647, -0.3680],
                                [ 0.2293,  1.3255,  0.1318,  2.0501],
                                [ 0.4058, -0.6624, -0.8745,  0.7203],
                                [ 0.5582,  0.0786, -0.6817,  0.6902],
                                [ 0.4309, -1.3067, -0.8823,  1.5977]]).float()
embedding.weight = nn.Parameter(custom_weights)
print(embedding.weight)

Parameter containing:
tensor([[-0.1882,  0.5530,  1.6267,  0.7013],
        [ 1.7840, -0.8278, -0.2701,  1.3586],
        [ 1.0281, -1.9094,  0.3182,  0.4211],
        [-1.3083, -0.0987,  0.7647, -0.3680],
        [ 0.2293,  1.3255,  0.1318,  2.0501],
        [ 0.4058, -0.6624, -0.8745,  0.7203],
        [ 0.5582,  0.0786, -0.6817,  0.6902],
        [ 0.4309, -1.3067, -0.8823,  1.5977]], requires_grad=True)


In [14]:
data = torch.tensor([0, 4, 7, 2, 1], dtype=torch.long)
label = torch.tensor([1], dtype=torch.float)

x = embedding(data)
x = nn.Flatten(0)(x)
x = nn.Linear(20, 1)(x)
output = nn.Sigmoid()(x)

In [15]:
loss = nn.BCELoss()(output, label)
optimizer = torch.optim.Adam(embedding.parameters())
loss.backward()
optimizer.step()
print(embedding.weight)

Parameter containing:
tensor([[-0.1872,  0.5520,  1.6277,  0.7023],
        [ 1.7850, -0.8288, -0.2711,  1.3596],
        [ 1.0291, -1.9084,  0.3172,  0.4221],
        [-1.3083, -0.0987,  0.7647, -0.3680],
        [ 0.2303,  1.3265,  0.1308,  2.0511],
        [ 0.4058, -0.6624, -0.8745,  0.7203],
        [ 0.5582,  0.0786, -0.6817,  0.6902],
        [ 0.4299, -1.3057, -0.8833,  1.5987]], requires_grad=True)
